# **Distribución % de créditos por organismo**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "puebla" in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_puebla',
 'df_financiamientos_2020_puebla',
 'df_financiamientos_2021_puebla',
 'df_financiamientos_2022_puebla',
 'df_financiamientos_2023_puebla',
 'df_financiamientos_2024_puebla']

## 2022

In [2]:
finance = df_financiamientos_2022_puebla[['id','organismo']]
finance

,id,organismo
0,1,4
1,2,1
2,3,1
3,4,2
4,5,2
...,...,...
12158,12159,2
12159,12160,3
12160,12161,3
12161,12162,4


In [3]:
# Convertir la columna 'organismo' a cadena de texto (str)
#finance['organismo'] = finance['organismo'].astype(str)
finance.loc[:, 'organismo'] = finance['organismo'].astype(str)

# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'INFONAVIT',
    '2': 'CNBV',
    '3': 'FOVISSTE',
    '5': 'CONAVI',
    '7': 'BANJERCITO',
    '26': 'INSUS'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
#finance['organismo'] = finance['organismo'].replace(categorias)
finance.loc[:, 'organismo'] = finance['organismo'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
#finance['organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)
finance.loc[:, 'organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Eliminar la fila donde rango_ingresos es '0'
finance = finance[finance['organismo'] != '-']

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance['organismo'].value_counts().reset_index()
conteo_organismos.columns = ['organismo', 'conteo']


# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['organismo'].unique():
        nuevo_registro = pd.DataFrame({'organismo': [categoria], 'conteo': [0]})
        conteo_organismos = pd.concat([conteo_organismos, nuevo_registro], ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='organismo').reset_index(drop=True)

print(conteo_organismos)

    organismo  conteo
0  BANJERCITO      35
1        CNBV    4064
2      CONAVI     745
3    FOVISSTE    1237
4   INFONAVIT    5235
5       INSUS      40


C:\Users\yoe11\AppData\Local\Temp\ipykernel_9164\2664051192.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['4' '1' '1' ... '3' '4' '4']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  finance.loc[:, 'organismo'] = finance['organismo'].astype(str)


In [4]:
# Crear un DataFrame con el conteo de registros por cada organismo
df_conteo_organismos = finance['organismo'].value_counts().reset_index()
df_conteo_organismos.columns = ['organismo', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
df_conteo_organismos['promedio'] = round((df_conteo_organismos['conteo'] * 100) / total_registros,1)
df_conteo_organismos

,organismo,conteo,promedio
0,INFONAVIT,5235,46.1
1,CNBV,4064,35.8
2,FOVISSTE,1237,10.9
3,CONAVI,745,6.6
4,INSUS,40,0.4
5,BANJERCITO,35,0.3


In [5]:
# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
conteo_organismos['promedio'] = round((conteo_organismos['conteo'] * 100) / total_registros,1)
conteo_organismos

,organismo,conteo,promedio
0,BANJERCITO,35,0.3
1,CNBV,4064,35.8
2,CONAVI,745,6.6
3,FOVISSTE,1237,10.9
4,INFONAVIT,5235,46.1
5,INSUS,40,0.4


In [6]:
f_2022 = finance.size
cont = df_conteo_organismos['conteo'].sum()
print("Hay ", f_2022, " registros en la base")
print("Hay ", cont, " registros al contar")

Hay  22712  registros en la base
Hay  11356  registros al contar


In [7]:
# Definir el diccionario de orden de organismos
orden_organismos = {
    'INFONAVIT': 1,
    'CNBV': 2,
    'FOVISSTE': 3,
    'BANJERCITO': 4
}

#df_conteo_organismos['orden'] = df_conteo_organismos['organismo'].map(orden_organismos)
df_conteo_organismos.loc[:, 'orden'] = df_conteo_organismos['organismo'].map(orden_organismos)

# Ordenar el DataFrame por el orden de los organismos
df_conteo_organismos = df_conteo_organismos.sort_values(by='orden')
df_conteo_organismos = df_conteo_organismos.drop(columns=['orden'])
df_conteo_organismos

,organismo,conteo,promedio
0,INFONAVIT,5235,46.1
1,CNBV,4064,35.8
2,FOVISSTE,1237,10.9
5,BANJERCITO,35,0.3
3,CONAVI,745,6.6
4,INSUS,40,0.4


In [8]:
df_conteo_organismos['promedio'].sum()

np.float64(100.10000000000001)

In [9]:

labels = df_conteo_organismos['organismo']
values = df_conteo_organismos['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos por organismo (2022)',
    #title_font_size = 22,
    xaxis=dict(title="Organismo"),
    yaxis=dict(
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_creditos_organismo_2022', carpeta='assets\graficas')
fig.show()

<>:31: SyntaxWarning: invalid escape sequence '\g'
<>:47: SyntaxWarning: invalid escape sequence '\g'
<>:31: SyntaxWarning: invalid escape sequence '\g'
<>:47: SyntaxWarning: invalid escape sequence '\g'
C:\Users\yoe11\AppData\Local\Temp\ipykernel_9164\2896638640.py:31: SyntaxWarning: invalid escape sequence '\g'
  def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
C:\Users\yoe11\AppData\Local\Temp\ipykernel_9164\2896638640.py:47: SyntaxWarning: invalid escape sequence '\g'
  guardar_grafico_como_html(fig, 'g_bar_creditos_organismo_2022', carpeta='assets\graficas')


## 2023

In [10]:
finance = df_financiamientos_2023_puebla[['id','organismo']]
finance

,id,organismo
0,1,4
1,2,1
2,3,1
3,4,2
4,5,2
...,...,...
10832,10833,1
10833,10834,4
10834,10835,4
10835,10836,4


In [11]:
# Convertir la columna 'organismo' a cadena de texto (str)
#finance['organismo'] = finance['organismo'].astype(str)
finance.loc[:, 'organismo'] = finance['organismo'].astype(str)

# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'INFONAVIT',
    '2': 'CNBV',
    '3': 'FOVISSTE',
    '5': 'CONAVI',
    '7': 'BANJERCITO',
    '26': 'INSUS'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
#finance['organismo'] = finance['organismo'].replace(categorias)
finance.loc[:, 'organismo'] = finance['organismo'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
#finance['organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)
finance.loc[:, 'organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Eliminar la fila donde rango_ingresos es '0'
finance = finance[finance['organismo'] != '-']

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance['organismo'].value_counts().reset_index()
conteo_organismos.columns = ['organismo', 'conteo']


# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['organismo'].unique():
        nuevo_registro = pd.DataFrame({'organismo': [categoria], 'conteo': [0]})
        conteo_organismos = pd.concat([conteo_organismos, nuevo_registro], ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='organismo').reset_index(drop=True)

print(conteo_organismos)

    organismo  conteo
0  BANJERCITO      42
1        CNBV    3275
2      CONAVI     230
3    FOVISSTE     985
4   INFONAVIT    5654
5       INSUS      99


C:\Users\yoe11\AppData\Local\Temp\ipykernel_9164\2664051192.py:3: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['4' '1' '1' ... '4' '4' '4']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



In [12]:
# Crear un DataFrame con el conteo de registros por cada organismo
df_conteo_organismos = finance['organismo'].value_counts().reset_index()
df_conteo_organismos.columns = ['organismo', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
df_conteo_organismos['promedio'] = round((df_conteo_organismos['conteo'] * 100) / total_registros,1)
df_conteo_organismos

,organismo,conteo,promedio
0,INFONAVIT,5654,55.0
1,CNBV,3275,31.8
2,FOVISSTE,985,9.6
3,CONAVI,230,2.2
4,INSUS,99,1.0
5,BANJERCITO,42,0.4


In [13]:
# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
conteo_organismos['promedio'] = round((conteo_organismos['conteo'] * 100) / total_registros,1)
conteo_organismos

,organismo,conteo,promedio
0,BANJERCITO,42,0.4
1,CNBV,3275,31.8
2,CONAVI,230,2.2
3,FOVISSTE,985,9.6
4,INFONAVIT,5654,55.0
5,INSUS,99,1.0


In [14]:
f_2022 = finance.size
cont = df_conteo_organismos['conteo'].sum()
print("Hay ", f_2022, " registros en la base")
print("Hay ", cont, " registros al contar")

Hay  20570  registros en la base
Hay  10285  registros al contar


In [15]:
# Definir el diccionario de orden de organismos
orden_organismos = {
    'INFONAVIT': 1,
    'CNBV': 2,
    'FOVISSTE': 3,
    'BANJERCITO': 4
}

#df_conteo_organismos['orden'] = df_conteo_organismos['organismo'].map(orden_organismos)
df_conteo_organismos.loc[:, 'orden'] = df_conteo_organismos['organismo'].map(orden_organismos)

# Ordenar el DataFrame por el orden de los organismos
df_conteo_organismos = df_conteo_organismos.sort_values(by='orden')
df_conteo_organismos = df_conteo_organismos.drop(columns=['orden'])
df_conteo_organismos

,organismo,conteo,promedio
0,INFONAVIT,5654,55.0
1,CNBV,3275,31.8
2,FOVISSTE,985,9.6
5,BANJERCITO,42,0.4
3,CONAVI,230,2.2
4,INSUS,99,1.0


In [16]:
df_conteo_organismos['promedio'].sum()

np.float64(100.0)

In [17]:

labels = df_conteo_organismos['organismo']
values = df_conteo_organismos['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos por organismo (2022)',
    #title_font_size = 22,
    xaxis=dict(title="Organismo"),
    yaxis=dict(
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_creditos_organismo_2023', carpeta='assets\graficas')
fig.show()

<>:31: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

<>:31: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_9164\2028246770.py:31: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_9164\2028246770.py:47: SyntaxWarning:

invalid escape sequence '\g'



## 2024

In [18]:
finance = df_financiamientos_2024_puebla[['id','organismo']]
finance

,id,organismo
0,1,4
1,2,2
2,3,4
3,4,4
4,5,4
...,...,...
6529,6530,2
6530,6531,4
6531,6532,4
6532,6533,1


In [19]:
# Convertir la columna 'organismo' a cadena de texto (str)
#finance['organismo'] = finance['organismo'].astype(str)
finance.loc[:, 'organismo'] = finance['organismo'].astype(str)

# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'INFONAVIT',
    '2': 'CNBV',
    '3': 'FOVISSTE',
    '5': 'CONAVI',
    '7': 'BANJERCITO',
    '26': 'INSUS'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
#finance['organismo'] = finance['organismo'].replace(categorias)
finance.loc[:, 'organismo'] = finance['organismo'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
#finance['organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)
finance.loc[:, 'organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Eliminar la fila donde rango_ingresos es '0'
finance = finance[finance['organismo'] != '-']

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance['organismo'].value_counts().reset_index()
conteo_organismos.columns = ['organismo', 'conteo']


# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['organismo'].unique():
        nuevo_registro = pd.DataFrame({'organismo': [categoria], 'conteo': [0]})
        conteo_organismos = pd.concat([conteo_organismos, nuevo_registro], ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='organismo').reset_index(drop=True)

print(conteo_organismos)

    organismo  conteo
0  BANJERCITO      20
1        CNBV    1706
2      CONAVI      32
3    FOVISSTE     477
4   INFONAVIT    3524
5       INSUS      16


C:\Users\yoe11\AppData\Local\Temp\ipykernel_9164\2664051192.py:3: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['4' '2' '4' ... '4' '1' '1']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



In [20]:
# Crear un DataFrame con el conteo de registros por cada organismo
df_conteo_organismos = finance['organismo'].value_counts().reset_index()
df_conteo_organismos.columns = ['organismo', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
df_conteo_organismos['promedio'] = round((df_conteo_organismos['conteo'] * 100) / total_registros,1)
df_conteo_organismos

,organismo,conteo,promedio
0,INFONAVIT,3524,61.0
1,CNBV,1706,29.5
2,FOVISSTE,477,8.3
3,CONAVI,32,0.6
4,BANJERCITO,20,0.3
5,INSUS,16,0.3


In [21]:
# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
conteo_organismos['promedio'] = round((conteo_organismos['conteo'] * 100) / total_registros,1)
conteo_organismos

,organismo,conteo,promedio
0,BANJERCITO,20,0.3
1,CNBV,1706,29.5
2,CONAVI,32,0.6
3,FOVISSTE,477,8.3
4,INFONAVIT,3524,61.0
5,INSUS,16,0.3


In [22]:
f_2022 = finance.size
cont = df_conteo_organismos['conteo'].sum()
print("Hay ", f_2022, " registros en la base")
print("Hay ", cont, " registros al contar")

Hay  11550  registros en la base
Hay  5775  registros al contar


In [23]:
# Definir el diccionario de orden de organismos
orden_organismos = {
    'INFONAVIT': 1,
    'CNBV': 2,
    'FOVISSTE': 3,
    'BANJERCITO': 4
}

#df_conteo_organismos['orden'] = df_conteo_organismos['organismo'].map(orden_organismos)
df_conteo_organismos.loc[:, 'orden'] = df_conteo_organismos['organismo'].map(orden_organismos)

# Ordenar el DataFrame por el orden de los organismos
df_conteo_organismos = df_conteo_organismos.sort_values(by='orden')
df_conteo_organismos = df_conteo_organismos.drop(columns=['orden'])
df_conteo_organismos

,organismo,conteo,promedio
0,INFONAVIT,3524,61.0
1,CNBV,1706,29.5
2,FOVISSTE,477,8.3
4,BANJERCITO,20,0.3
3,CONAVI,32,0.6
5,INSUS,16,0.3


In [24]:
df_conteo_organismos['promedio'].sum()

np.float64(99.99999999999999)

In [25]:

labels = df_conteo_organismos['organismo']
values = df_conteo_organismos['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos por organismo (2022)',
    #title_font_size = 22,
    xaxis=dict(title="Organismo"),
    yaxis=dict(
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_creditos_organismo_2024', carpeta='assets\graficas')
fig.show()

<>:31: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

<>:31: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_9164\2348481071.py:31: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_9164\2348481071.py:47: SyntaxWarning:

invalid escape sequence '\g'

